# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
import json

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,bukama,-9.2000,25.8500,21.39,98,100,2.07,CD,1702065123
1,1,enewetak,11.3474,162.3373,27.46,77,86,9.95,MH,1702065123
2,2,ribeira grande,38.5167,-28.7000,18.22,88,75,7.72,PT,1702065124
3,3,tongouson,5.0214,120.1328,27.30,81,21,2.71,PH,1702065124
4,4,qamdo,31.1667,97.2333,-7.29,74,3,2.34,CN,1702065124


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

# Configure the map plot
cities_map = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    size = "Humidity",
    color = "City"
)

# Display the map
cities_map

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_cities_df = city_data_df.loc[(city_data_df['Max Temp'] <= 26.67)]

# Drop any rows with null values
ideal_cities_df = ideal_cities_df.dropna()

# Display sample data
ideal_cities_df


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,bukama,-9.2000,25.8500,21.39,98,100,2.07,CD,1702065123
2,2,ribeira grande,38.5167,-28.7000,18.22,88,75,7.72,PT,1702065124
4,4,qamdo,31.1667,97.2333,-7.29,74,3,2.34,CN,1702065124
5,5,katsuura,35.1333,140.3000,13.79,71,0,5.43,JP,1702065125
6,6,anadyr,64.7500,177.4833,-23.72,77,0,4.00,RU,1702065125
...,...,...,...,...,...,...,...,...,...,...
538,538,louga,58.7372,29.8453,-9.29,93,100,3.77,RU,1702065262
542,542,kuanda,56.3161,116.0806,-34.76,93,49,0.59,RU,1702065263
543,543,fayetteville,35.0527,-78.8784,18.61,30,0,4.63,US,1702065149
544,544,kadipur,26.1667,82.3833,17.72,51,2,2.53,IN,1702065263


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_cities_df[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
# YOUR CODE HERE
hotel_df['Hotel Name'] = ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
0,bukama,CD,-9.2000,25.8500,98,
2,ribeira grande,PT,38.5167,-28.7000,88,
4,qamdo,CN,31.1667,97.2333,74,
5,katsuura,JP,35.1333,140.3000,71,
6,anadyr,RU,64.7500,177.4833,77,
...,...,...,...,...,...,...
538,louga,RU,58.7372,29.8453,93,
542,kuanda,RU,56.3161,116.0806,93,
543,fayetteville,US,35.0527,-78.8784,30,
544,kadipur,IN,26.1667,82.3833,51,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [6]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "apiKey":geoapify_key,
    "categories":"accommodation.hotel"
}

# Print a message to follow up the hotel search
print("Starting hotel search")


# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    latitude = hotel_df.loc[index,'Lat']
    longitude = hotel_df.loc[index,'Lng']
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params['filter'] = f"circle:{longitude},{latitude},{radius}"
    params['bias'] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"
    

    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)

    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
bukama - nearest hotel: Mission catholique
ribeira grande - nearest hotel: Hospedaria JSF
qamdo - nearest hotel: 银杏商务宾馆
katsuura - nearest hotel: 潮騒の宿
anadyr - nearest hotel: Гостевой дом
jayapura - nearest hotel: Hotel Jasmine Jayapura
adamstown - nearest hotel: No hotel found
ushuaia - nearest hotel: Apart Hotel Aires del Beagle
island harbour - nearest hotel: Premier Inn
campo verde - nearest hotel: No hotel found
edinburgh of the seven seas - nearest hotel: No hotel found
badger - nearest hotel: Hotel North Pole
santiago de cao - nearest hotel: Hostal El Encanto
yellowknife - nearest hotel: No hotel found
port-aux-francais - nearest hotel: Keravel
tiksi - nearest hotel: Арктика
tyukhtet - nearest hotel: No hotel found
blackmans bay - nearest hotel: Villa Howden
holualoa - nearest hotel: Kona Hotel
chiredzi - nearest hotel: Nesbitt Arms Hotel
berdigestyakh - nearest hotel: No hotel found
stanley - nearest hotel: Hotel 52
kapa'a - nearest hotel: Pono Kai Resort


sanctuary point - nearest hotel: Dungowan Accommodation
cill airne - nearest hotel: The Parkavon Hotel
shira - nearest hotel: No hotel found
hundested - nearest hotel: Lynæs hotel og selvskabslokaler
ancud - nearest hotel: Cabañas
gotsucho - nearest hotel: No hotel found
kirs - nearest hotel: No hotel found
waegwan - nearest hotel: No hotel found
kruisfontein - nearest hotel: No hotel found
gorom-gorom - nearest hotel: No hotel found
bodaybo - nearest hotel: НОЙ
timaru - nearest hotel: The Grosvenor Hotel
nago - nearest hotel: グリーンリッチホテル
zhaotong - nearest hotel: Autumn Hotel
ghat - nearest hotel: فندق تاسيلي
toamasina - nearest hotel: Androranga
kapuskasing - nearest hotel: Travelodge
kendari - nearest hotel: City Hotel Kendari
yemetsk - nearest hotel: No hotel found
saint-joseph - nearest hotel: PALM Hotel & Spa
cacapava do sul - nearest hotel: Hotel Portal do Pampa
kerikeri - nearest hotel: Avalon Resort
vadso - nearest hotel: Scandic Vadsø
kharkhorin - nearest hotel: 점심식사 good rest

treinta y tres - nearest hotel: No hotel found
lumajang - nearest hotel: Hotel Gajah Mada
bamboo flat - nearest hotel: Hotel Megapode Nest
jampur - nearest hotel: City Marriage Hall
raja - nearest hotel: No hotel found
aioun - nearest hotel: فندق العيون
tuy hoa - nearest hotel: Vinh Tuan Hotel
waat - nearest hotel: Hotel Montana
luanda - nearest hotel: Hotel Forum
coquimbo - nearest hotel: Hotel Iberia
pacific grove - nearest hotel: Pacific Grove Inn
al jawf - nearest hotel: No hotel found
muynoq - nearest hotel: Lala
naama - nearest hotel: Résidence des hôtes إقامة الضيوف
santa elena de uairen - nearest hotel: Posada Villa Fairmont
sabzevar - nearest hotel: مامورسرا مخابرات سبزوار
dikhil - nearest hotel: No hotel found
yuxi - nearest hotel: 宏盛酒店 A座
hita - nearest hotel: No hotel found
chingirlau - nearest hotel: No hotel found
kinablangan - nearest hotel: No hotel found
collie - nearest hotel: Federal Hotel
yenifakili - nearest hotel: No hotel found
duayaw-nkwanta - nearest hotel: No 

,City,Country,Lat,Lng,Humidity,Hotel Name
0,bukama,CD,-9.2000,25.8500,98,Mission catholique
2,ribeira grande,PT,38.5167,-28.7000,88,Hospedaria JSF
4,qamdo,CN,31.1667,97.2333,74,银杏商务宾馆
5,katsuura,JP,35.1333,140.3000,71,潮騒の宿
6,anadyr,RU,64.7500,177.4833,77,Гостевой дом
...,...,...,...,...,...,...
538,louga,RU,58.7372,29.8453,93,Луга
542,kuanda,RU,56.3161,116.0806,93,No hotel found
543,fayetteville,US,35.0527,-78.8784,30,Regency Inn
544,kadipur,IN,26.1667,82.3833,51,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [7]:
%%capture --no-display

# Configure the map plot
hotels_map = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    size = "Humidity",
    color = "City",
    hover_cols = ["Hotel Name", "Country"]
)

# Display the map
hotels_map

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)